# Weight predictor 

In [16]:
import numpy as np
import pandas as pd
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, RidgeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor


In [17]:
try:
    df = pd.read_csv(r"/workspaces/ML-Journey/Datasets/Dataset for weight prediction with gender.csv")
except:
    df = pd.read_csv(r"C:\\Users\\Archit\\Documents\\GitHub\\ML-Journey\\Datasets\\Dataset for weight prediction with gender.csv")

In [18]:
X = df.drop(columns=['weight_kg'])
y = df[['weight_kg']]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [20]:
X_test

,height_cm,gender,bmi_class_Normal,bmi_class_Obesity,bmi_class_Overweight,bmi_class_Underweight
10635,167.4,0,0,0,1,0
5496,162.3,1,0,0,1,0
13055,178.5,1,0,0,1,0
11826,180.9,1,1,0,0,0
1326,177.3,1,0,0,1,0
...,...,...,...,...,...,...
1472,168.4,1,0,0,1,0
8030,152.2,0,1,0,0,0
12738,163.2,1,1,0,0,0
9960,172.6,1,0,0,1,0


In [21]:
scaler = StandardScaler()
X_train_trf = scaler.fit_transform(X_train)
X_test_trf = scaler.transform(X_test)

In [44]:
knn = KNeighborsRegressor(n_neighbors=15)
knn.fit(X_train_trf, y_train)
y_pred = knn.predict(X_test_trf)
r2_score(y_test, y_pred)

0.8765862708663774

In [71]:
elasticnet = ElasticNet(alpha=0.000001)
elasticnet.fit(X_train, y_train)
y_pred = elasticnet.predict(X_test)
r2_score(y_test, y_pred)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.180e+04, tolerance: 1.521e+02
  model = cd_fast.enet_coordinate_descent(


0.8761686232220378

In [91]:
dt = DecisionTreeRegressor(ccp_alpha=0.01)
dt.fit(X_train_trf, y_train)
y_pred = dt.predict(X_test_trf)
r2_score(y_test, y_pred)

0.8739919398446582

In [92]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.8761686024815241

In [93]:
ensemble = VotingRegressor(
    estimators=[
        ('linear regression', lr),
        ('dtree', dt),
        ('knn', knn),
        ('elastic net', elasticnet)
    ]
)

In [94]:
ensemble.fit(X_train_trf, y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/ensemble/_voting.py:694: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.669e+04, tolerance: 1.521e+02
  model = cd_fast.enet_coordinate_descent(


VotingRegressor(estimators=[('linear regression', LinearRegression()),
                            ('dtree', DecisionTreeRegressor(ccp_alpha=0.01)),
                            ('knn', KNeighborsRegressor(n_neighbors=15)),
                            ('elastic net', ElasticNet(alpha=1e-06))])

In [95]:
ensemble_pred = ensemble.predict(X_test_trf)

In [51]:
r2_score(y_test, ensemble_pred)

0.8804530518399114

In [99]:
height = 170
pa = [[0,0,1,0]]

ensemble.predict(scaler.transform([[167, 1, 1,0,0,0]]))

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([64.15327454])